## 건설업 기업 재무구조 휴리스틱 평가 모델 개발

In [5]:
# ## 0. 데이터 로드 및 전처리

# In[1]:


import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

# 데이터 로드
df = pd.read_csv('dart_final.csv')

print("=== 데이터 정보 ===")
print(f"데이터 shape: {df.shape}")
print(f"\n컬럼 목록 (총 {len(df.columns)}개):")
for i, col in enumerate(df.columns, 1):
    print(f"{i:2d}. {col}")

# 분기 정렬
qcat = pd.CategoricalDtype(categories=['Q1','Q2','Q3','Q4'], ordered=True)
df['quarter'] = df['quarter'].astype(qcat)
df = df.sort_values(['corp_name','year','quarter']).reset_index(drop=True)

# 모든 재무 지표를 숫자형으로 변환
financial_cols = ['부채비율', '자기자본비율', 'ROA', 'ROE', 
                  '매출액성장률', '영업이익성장률', '순이익성장률']

for col in financial_cols:
    df[col] = pd.to_numeric(df[col], errors='coerce')

print("\n=== 데이터 타입 확인 ===")
print(df.dtypes)

print("\n=== 데이터 미리보기 ===")
df.head()

=== 데이터 정보 ===
데이터 shape: (1140, 16)

컬럼 목록 (총 16개):
 1. corp_name
 2. year
 3. quarter
 4. 자산총계
 5. 부채총계
 6. 자본총계
 7. 매출액
 8. 영업이익
 9. 분기순이익
10. 부채비율
11. 자기자본비율
12. ROA
13. ROE
14. 매출액성장률
15. 영업이익성장률
16. 순이익성장률

=== 데이터 타입 확인 ===
corp_name      object
year            int64
quarter      category
자산총계          float64
부채총계          float64
자본총계          float64
매출액           float64
영업이익          float64
분기순이익         float64
부채비율          float64
자기자본비율        float64
ROA           float64
ROE           float64
매출액성장률        float64
영업이익성장률       float64
순이익성장률        float64
dtype: object

=== 데이터 미리보기 ===


,corp_name,year,quarter,자산총계,부채총계,자본총계,매출액,영업이익,분기순이익,부채비율,자기자본비율,ROA,ROE,매출액성장률,영업이익성장률,순이익성장률
0,DL이앤씨,2016,Q2,1.237828e+13,7.324959e+12,5.053324e+12,2.563786e+12,1.361732e+11,1.197966e+11,144.953291,40.824110,0.967797,2.370650,13.758516,50.010346,285.979110
1,DL이앤씨,2016,Q3,1.218542e+13,7.042494e+12,5.142928e+12,2.457364e+12,1.306544e+11,1.090875e+11,136.935499,42.205579,0.895230,2.121117,-4.150984,-4.052776,-8.939388
2,DL이앤씨,2016,Q4,1.239151e+13,7.246135e+12,5.145374e+12,2.578911e+12,6.178428e+10,3.327409e+10,140.828150,41.523385,0.268523,0.646680,4.946246,-52.711673,-69.497809
3,DL이앤씨,2017,Q1,1.281212e+13,7.563966e+12,5.248157e+12,2.511359e+12,1.139837e+11,1.493461e+11,144.126142,40.962430,1.165663,2.845688,-2.619402,84.486574,348.836136
4,DL이앤씨,2017,Q2,1.320628e+13,7.818788e+12,5.387496e+12,3.106287e+12,1.430366e+11,1.045337e+11,145.128425,40.794942,0.791545,1.940302,23.689504,25.488610,-30.005759


In [6]:
# ## a. 위험 플래그 계산 (0과 1로 세우기)

# In[2]:


# 그룹화 객체 생성
g = df.groupby('corp_name', group_keys=False)

# 1) 완전자본잠식
df['완전자본잠식'] = (df['자본총계'] < 0).astype(int)

# 2) 연속 매출액 성장률 감소 (8분기 연속)
df['연속_매출액성장률_감소'] = g['매출액성장률'].transform(
    lambda s: (s < 0).astype('int8').rolling(8, min_periods=8).sum()
).eq(8).astype(int)

# 3) 부채비율 300% 이상
df['부채비율_300이상'] = (df['부채비율'] >= 300).astype(int)

# 4) 영업손실 4분기 연속 발생
df['영업손실_4분기연속'] = g['영업이익'].transform(
    lambda s: (s < 0).astype('int8').rolling(4, min_periods=4).sum()
).eq(4).astype(int)

# 5) 영업이익 성장률 8분기 연속 감소
df['영업이익성장률_8분기연속감소'] = g['영업이익성장률'].transform(
    lambda s: (s < 0).astype('int8').rolling(8, min_periods=8).sum()
).eq(8).astype(int)

# 위험 지표 컬럼 리스트
risk_cols = ['완전자본잠식','연속_매출액성장률_감소','부채비율_300이상',
             '영업손실_4분기연속','영업이익성장률_8분기연속감소']

# 각 행(기업-분기)별 총 위험 플래그 개수 계산
df['위험플래그_개수'] = df[risk_cols].sum(axis=1)

print("=== 위험 플래그 계산 완료 ===")
print("\n기업-분기별 위험 플래그 예시 (삼성물산):")
sample_corp = df[df['corp_name'] == '삼성물산'][['corp_name','year','quarter'] + risk_cols + ['위험플래그_개수']].head(8)
print(sample_corp)

print("\n=== 전체 위험 플래그 분포 ===")
print(df['위험플래그_개수'].value_counts().sort_index())


=== 위험 플래그 계산 완료 ===

기업-분기별 위험 플래그 예시 (삼성물산):
    corp_name  year quarter  완전자본잠식  연속_매출액성장률_감소  부채비율_300이상  영업손실_4분기연속  \
611      삼성물산  2016      Q2       0             0           0           0   
612      삼성물산  2016      Q3       0             0           0           0   
613      삼성물산  2016      Q4       0             0           0           0   
614      삼성물산  2017      Q1       0             0           0           0   
615      삼성물산  2017      Q2       0             0           0           0   
616      삼성물산  2017      Q3       0             0           0           0   
617      삼성물산  2017      Q4       0             0           0           0   
618      삼성물산  2018      Q1       0             0           0           0   

     영업이익성장률_8분기연속감소  위험플래그_개수  
611                0         0  
612                0         0  
613                0         0  
614                0         0  
615                0         0  
616                0         0  
617                0        

In [7]:
# ## b. 라벨링 (0: 우수, 1: 보통, 2: 위험)

# In[3]:


# 라벨링 함수
def assign_risk_label(flag_count):
    if flag_count <= 1:
        return 0  # 우수
    elif flag_count <= 3:
        return 1  # 보통
    else:
        return 2  # 위험

# 라벨 적용
df['risk_label'] = df['위험플래그_개수'].apply(assign_risk_label)

# 라벨 명칭 추가 (시각화용)
label_names = {0: '우수', 1: '보통', 2: '위험'}
df['risk_label_name'] = df['risk_label'].map(label_names)

print("=== 라벨링 완료 ===")
print("\n라벨별 분포:")
print(df['risk_label_name'].value_counts())

print("\n라벨링 예시 (기업-분기별):")
print(df[['corp_name','year','quarter','위험플래그_개수','risk_label','risk_label_name']].head(20))


=== 라벨링 완료 ===

라벨별 분포:
risk_label_name
우수    1130
보통      10
Name: count, dtype: int64

라벨링 예시 (기업-분기별):
   corp_name  year quarter  위험플래그_개수  risk_label risk_label_name
0      DL이앤씨  2016      Q2         0           0              우수
1      DL이앤씨  2016      Q3         0           0              우수
2      DL이앤씨  2016      Q4         0           0              우수
3      DL이앤씨  2017      Q1         0           0              우수
4      DL이앤씨  2017      Q2         0           0              우수
5      DL이앤씨  2017      Q3         0           0              우수
6      DL이앤씨  2017      Q4         0           0              우수
7      DL이앤씨  2018      Q1         0           0              우수
8      DL이앤씨  2018      Q2         0           0              우수
9      DL이앤씨  2018      Q3         0           0              우수
10     DL이앤씨  2018      Q4         0           0              우수
11     DL이앤씨  2019      Q1         0           0              우수
12     DL이앤씨  2019      Q2         0           0 

In [8]:
# ## c. 휴리스틱 모델 구성을 위한 재무지표 선정

# In[4]:


# 모델에 사용할 재무지표 선정
feature_cols = ['부채비율', '자기자본비율', 'ROA', 'ROE', 
                '매출액성장률', '영업이익성장률', '순이익성장률']

# 사용 가능한 컬럼만 선택
available_features = [col for col in feature_cols if col in df.columns]
print(f"사용 가능한 재무지표: {available_features}")

# 특징 데이터 정규화 (스케일링)
from sklearn.preprocessing import StandardScaler

# NaN 처리 및 정규화
df_features = df[available_features].fillna(0)
scaler = StandardScaler()
df_scaled = pd.DataFrame(
    scaler.fit_transform(df_features),
    columns=[f'{col}_scaled' for col in available_features],
    index=df.index
)

# 원본 데이터프레임에 추가
df = pd.concat([df, df_scaled], axis=1)

print("\n정규화된 특징 데이터 미리보기:")
print(df[[f'{col}_scaled' for col in available_features[:3]]].head())


사용 가능한 재무지표: ['부채비율', '자기자본비율', 'ROA', 'ROE', '매출액성장률', '영업이익성장률', '순이익성장률']

정규화된 특징 데이터 미리보기:
   부채비율_scaled  자기자본비율_scaled  ROA_scaled
0    -0.215190      -0.016295    0.125341
1    -0.250228       0.057903    0.102069
2    -0.233217       0.021263   -0.098918
3    -0.218804      -0.008866    0.188798
4    -0.214424      -0.017861    0.068817

정규화된 특징 데이터 미리보기:
   부채비율_scaled  자기자본비율_scaled  ROA_scaled
0    -0.215190      -0.016295    0.125341
1    -0.250228       0.057903    0.102069
2    -0.233217       0.021263   -0.098918
3    -0.218804      -0.008866    0.188798
4    -0.214424      -0.017861    0.068817


In [9]:
# ## d. 다음 분기 예측을 위한 데이터 준비

# In[5]:


# 다음 분기 라벨 생성 (shift 사용)
df_sorted = df.sort_values(['corp_name', 'year', 'quarter']).reset_index(drop=True)

# 같은 기업 내에서만 shift
df_sorted['next_risk_label'] = df_sorted.groupby('corp_name')['risk_label'].shift(-1)

# 다음 분기 데이터가 있는 행만 선택 (학습용)
train_df = df_sorted[df_sorted['next_risk_label'].notna()].copy()

print(f"전체 데이터: {len(df_sorted)}개 행")
print(f"학습 가능 데이터 (다음 분기 존재): {len(train_df)}개 행")

print("\n데이터 예시 (현재 분기 -> 다음 분기 라벨):")
sample = train_df[['corp_name','year','quarter','risk_label','next_risk_label']].head(10)
print(sample)

전체 데이터: 1140개 행
학습 가능 데이터 (다음 분기 존재): 1108개 행

데이터 예시 (현재 분기 -> 다음 분기 라벨):
  corp_name  year quarter  risk_label  next_risk_label
0     DL이앤씨  2016      Q2           0              0.0
1     DL이앤씨  2016      Q3           0              0.0
2     DL이앤씨  2016      Q4           0              0.0
3     DL이앤씨  2017      Q1           0              0.0
4     DL이앤씨  2017      Q2           0              0.0
5     DL이앤씨  2017      Q3           0              0.0
6     DL이앤씨  2017      Q4           0              0.0
7     DL이앤씨  2018      Q1           0              0.0
8     DL이앤씨  2018      Q2           0              0.0
9     DL이앤씨  2018      Q3           0              0.0


In [10]:
# ## e. 휴리스틱 모델 가중치 최적화

# In[6]:


# 수동 경사하강법을 이용한 가중치 최적화
class HeuristicModel:
    def __init__(self, n_features):
        # 가중치 초기화
        self.weights = np.random.randn(n_features) * 0.01
        self.bias = 0
        self.history = []
        
    def predict(self, X):
        """선형 조합 계산"""
        return np.dot(X, self.weights) + self.bias
    
    def sigmoid(self, x):
        """시그모이드 함수 (0~1 범위)"""
        return 1 / (1 + np.exp(-np.clip(x, -500, 500)))
    
    def loss(self, y_true, y_pred):
        """평균 제곱 오차"""
        return np.mean((y_true - y_pred) ** 2)
    
    def train(self, X, y, learning_rate=0.01, epochs=100):
        """가중치 학습"""
        n_samples = X.shape[0]
        
        for epoch in range(epochs):
            # 순전파
            y_pred_raw = self.predict(X)
            y_pred = self.sigmoid(y_pred_raw) * 2  # 0~2 범위로 스케일링
            
            # 손실 계산
            current_loss = self.loss(y, y_pred)
            self.history.append(current_loss)
            
            # 역전파 (경사 계산)
            dy = -2 * (y - y_pred) / n_samples
            
            # 시그모이드 미분
            sigmoid_grad = self.sigmoid(y_pred_raw) * (1 - self.sigmoid(y_pred_raw))
            dy = dy * sigmoid_grad * 2
            
            # 가중치 업데이트
            dw = np.dot(X.T, dy)
            db = np.mean(dy)
            
            self.weights -= learning_rate * dw
            self.bias -= learning_rate * db
            
            if epoch % 20 == 0:
                print(f"Epoch {epoch:3d}, Loss: {current_loss:.6f}")
    
    def get_risk_score(self, X):
        """위험 점수 계산 (0~100)"""
        raw_score = self.predict(X)
        return np.clip(self.sigmoid(raw_score) * 100, 0, 100)

# 학습 데이터 준비
X_train = train_df[[f'{col}_scaled' for col in available_features]].values
y_train = train_df['next_risk_label'].values

# 모델 학습
model = HeuristicModel(n_features=len(available_features))
print("=== 휴리스틱 모델 학습 시작 ===")
model.train(X_train, y_train, learning_rate=0.001, epochs=100)

print("\n=== 학습된 가중치 ===")
for i, col in enumerate(available_features):
    print(f"{col}: {model.weights[i]:.4f}")
print(f"Bias: {model.bias:.4f}")


=== 휴리스틱 모델 학습 시작 ===
Epoch   0, Loss: 0.991050
Epoch  20, Loss: 0.990981
Epoch  40, Loss: 0.990913
Epoch  60, Loss: 0.990847
Epoch  80, Loss: 0.990782

=== 학습된 가중치 ===
부채비율: -0.0070
자기자본비율: -0.0008
ROA: -0.0034
ROE: -0.0020
매출액성장률: 0.0029
영업이익성장률: 0.0063
순이익성장률: -0.0062
Bias: -0.0001


In [11]:
# ## f. 위험 점수 계산 및 등급 분류 (Red/Yellow/Green)

# In[7]:


# 전체 데이터에 대해 위험 점수 계산
X_all = df_sorted[[f'{col}_scaled' for col in available_features]].values
df_sorted['risk_score'] = model.get_risk_score(X_all)

# Threshold 기반 등급 분류
def classify_risk_level(score):
    if score >= 70:
        return 'Red (위험)'
    elif score >= 40:
        return 'Yellow (주의)'
    else:
        return 'Green (안전)'

df_sorted['risk_level'] = df_sorted['risk_score'].apply(classify_risk_level)

print("=== 위험 점수 및 등급 분류 완료 ===")
print("\n위험 등급 분포:")
print(df_sorted['risk_level'].value_counts())

print("\n위험 점수 통계:")
print(df_sorted['risk_score'].describe())

print("\n예시 결과 (최근 분기):")
recent_sample = df_sorted[df_sorted['year'] >= 2024][
    ['corp_name','year','quarter','위험플래그_개수','risk_label_name','risk_score','risk_level']
].head(20)
print(recent_sample)

=== 위험 점수 및 등급 분류 완료 ===

위험 등급 분포:
risk_level
Yellow (주의)    1140
Name: count, dtype: int64

위험 점수 통계:
count    1140.000000
mean       49.997742
std         0.284827
min        48.158125
25%        49.930417
50%        49.993707
75%        50.049739
max        54.540593
Name: risk_score, dtype: float64

예시 결과 (최근 분기):
     corp_name  year quarter  위험플래그_개수 risk_label_name  risk_score  \
31       DL이앤씨  2024      Q1         0              우수   50.006118   
32       DL이앤씨  2024      Q2         0              우수   50.060650   
33       DL이앤씨  2024      Q3         0              우수   50.050714   
34       DL이앤씨  2024      Q4         0              우수   50.078102   
35       DL이앤씨  2025      Q1         0              우수   50.001089   
36       DL이앤씨  2025      Q2         0              우수   50.090262   
68        GS건설  2024      Q1         0              우수   49.910000   
69        GS건설  2024      Q2         0              우수   49.982732   
70        GS건설  2024      Q3         0           

In [16]:
# ## 모델 검증 및 성능 평가

# In[8]:


# 실제 라벨과 예측 등급 비교
def score_to_label(score):
    if score >= 70:
        return 2  # 위험
    elif score >= 40:
        return 1  # 보통
    else:
        return 0  # 우수

# 검증 데이터에서만 평가
val_df = train_df.copy()

# train_df에 risk_score 추가 (df_sorted에서 병합)
val_df = val_df.merge(
    df_sorted[['corp_name', 'year', 'quarter', 'risk_score']], 
    on=['corp_name', 'year', 'quarter'], 
    how='left'
)

val_df['predicted_label'] = val_df['risk_score'].apply(score_to_label)

# 정확도 계산
accuracy = (val_df['predicted_label'] == val_df['next_risk_label']).mean()
print(f"예측 정확도: {accuracy:.2%}")

# 혼동 행렬
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(val_df['next_risk_label'], val_df['predicted_label'], labels=[0, 1, 2])
print("\n혼동 행렬:")
print(pd.DataFrame(cm, 
                   index=['실제_우수','실제_보통','실제_위험'],
                   columns=['예측_우수','예측_보통','예측_위험']))

# 기업별 위험도 추이 예시
print("\n=== 특정 기업의 위험도 추이 ===")
sample_corp_name = df_sorted['corp_name'].iloc[0]
corp_trend = df_sorted[df_sorted['corp_name'] == sample_corp_name][
    ['year','quarter','risk_score','risk_level','위험플래그_개수']
].tail(8)
print(f"\n{sample_corp_name}의 최근 8분기 추이:")
print(corp_trend)


예측 정확도: 0.90%

혼동 행렬:
       예측_우수  예측_보통  예측_위험
실제_우수      0   1098      0
실제_보통      0     10      0
실제_위험      0      0      0

=== 특정 기업의 위험도 추이 ===

DL이앤씨의 최근 8분기 추이:
    year quarter  risk_score   risk_level  위험플래그_개수
29  2023      Q3   50.031883  Yellow (주의)         0
30  2023      Q4   50.119025  Yellow (주의)         0
31  2024      Q1   50.006118  Yellow (주의)         0
32  2024      Q2   50.060650  Yellow (주의)         0
33  2024      Q3   50.050714  Yellow (주의)         0
34  2024      Q4   50.078102  Yellow (주의)         0
35  2025      Q1   50.001089  Yellow (주의)         0
36  2025      Q2   50.090262  Yellow (주의)         0


In [17]:
# ## 최종 결과 저장

# In[9]:


# 최종 결과 저장
result_df = df_sorted[['corp_name','year','quarter','위험플래그_개수',
                       'risk_label','risk_label_name','risk_score','risk_level']]

# CSV로 저장
result_df.to_csv('construction_risk_evaluation_results.csv', index=False, encoding='utf-8-sig')

print("=== 최종 결과 저장 완료 ===")
print(f"저장 파일: construction_risk_evaluation_results.csv")
print(f"총 {len(result_df)}개 행 저장")

# 위험 기업 TOP 10
print("\n=== 현재 위험도 높은 기업 TOP 10 ===")
latest_quarter = result_df.groupby('corp_name').last().reset_index()
top_risk = latest_quarter.nlargest(10, 'risk_score')[
    ['corp_name','year','quarter','risk_score','risk_level']
]
print(top_risk)

=== 최종 결과 저장 완료 ===
저장 파일: construction_risk_evaluation_results.csv
총 1140개 행 저장

=== 현재 위험도 높은 기업 TOP 10 ===
    corp_name  year quarter  risk_score   risk_level
23       이수화학  2025      Q2   52.900788  Yellow (주의)
16       삼부토건  2025      Q2   52.442255  Yellow (주의)
25       진흥기업  2025      Q2   50.207172  Yellow (주의)
15       동원개발  2025      Q2   50.104969  Yellow (주의)
0       DL이앤씨  2025      Q2   50.090262  Yellow (주의)
22       우원개발  2025      Q2   50.088668  Yellow (주의)
2   HDC현대산업개발  2025      Q2   50.076553  Yellow (주의)
24     자이에스앤디  2025      Q2   50.070227  Yellow (주의)
20     신원종합개발  2025      Q2   50.068567  Yellow (주의)
17       삼성물산  2025      Q2   50.064338  Yellow (주의)
